# Xgboost

Проблема многих алгоримтов построения деревьев в том, что в них не уделяется должного внимания регуляризации. 
В классическом градиентном бустинге применяется такие меры:
- ограничение на структуру дерева: макисмальная глубина (max_depth), минимальное число объектов в листе (min_samples_leaf)
- контролирование темпа обучения (learning_rate)
- увеличение "непохожести" деревьев за счет рандомизации, как в случайном лесе

[Xgboost](https://github.com/dmlc/xgboost) использует еще больше параметров для регуляризации базовых деревьев.

Целевая функция для оптимизации в Xgboost состоит из двух слагаемых: специфичной пункции потерь и регуляризатора $\Omega (f_k)$ для каждого из $K$ деревьев, где $f_k$ - прогноз $k$-ого дерева.


$$
obj(\theta) = \sum_{i}^{\ell} l(y_i - \hat{y_i}) +  \sum_{k=1}^{K} \Omega (f_k)
$$

Функция потерь зависит от решаемой задачи (Xgboost адаптирован под задачи классификации, регрессии и ранжирования, подробней хорошо описано в [документации](http://xgboost.readthedocs.io/en/latest/model.html) Xgboost), а регуляризатор выглядит следующим образом:

$$
\Omega(f) = \gamma T + \frac{1}{2} \lambda \sum_{j=1}^{T}w_j^2
$$

Первое слагаемое ($\gamma T$) штрафует модель за большое число листьев $T$, а второе ($\frac{1}{2} \lambda \sum_{j=1}^{T}w_j^2$) контролирует сумму весов модели в листьях. 

# Примеры

В sklearn доступны алгоритмы AdaBoost и GradientBoosting для задач классификации и регрессии.
В качестве примера рассмотрим решение задачи восстановления одномерной регрессии с помощью <a href="http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html">GradientBoostingRegressor</a>.

In [15]:
import pandas as pd
import numpy as np
import xgboost as xgb
%matplotlib inline
import matplotlib.pyplot as plt

from sklearn.datasets import load_iris, load_boston
from sklearn.cross_validation import KFold
from sklearn.metrics import confusion_matrix, mean_squared_error

**Пример использования Xgboost для классификации на данных Iris.**

In [16]:
iris = load_iris()
X = iris['data']
y = iris['target']
kf = KFold(y.shape[0], n_folds=5, shuffle=True, random_state=13)
for train_index, test_index in kf:
    xgb_model = xgb.XGBClassifier().fit(X[train_index], y[train_index])
    predictions = xgb_model.predict(X[test_index])
    actuals = y[test_index]
    print(confusion_matrix(actuals, predictions))

[[ 9  0  0]
 [ 0  8  0]
 [ 0  2 11]]
[[10  0  0]
 [ 0  9  1]
 [ 0  1  9]]
[[9 0 0]
 [0 9 1]
 [0 2 9]]
[[12  0  0]
 [ 0 10  1]
 [ 0  0  7]]
[[10  0  0]
 [ 0 11  0]
 [ 0  0  9]]


**Пример восстановления регрессии с Xgboost на данных boston.**

In [17]:
boston = load_boston()
y = boston['target']
X = boston['data']
kf = KFold(y.shape[0], n_folds=5, shuffle=True, random_state=17)
for train_index, test_index in kf:
    xgb_model = xgb.XGBRegressor().fit(X[train_index],y[train_index])
    predictions = xgb_model.predict(X[test_index])
    actuals = y[test_index]
    print(mean_squared_error(actuals, predictions))

8.0971394504
8.92640647601
18.299771839
6.73819144817
8.47392580766


**Простой пример подбора параметров с GridSearchCV.**

In [18]:
from sklearn.grid_search import GridSearchCV
X = boston['data']
y = boston['target']

xgb_model = xgb.XGBRegressor()
xgb_grid = GridSearchCV(xgb_model,
                   {'max_depth': [2,4,6],
                    'n_estimators': [50,100,200]}, verbose=1)
xgb_grid.fit(X,y)
print(xgb_grid.best_score_)
print(xgb_grid.best_params_)

/usr/local/lib/python3.5/dist-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


Fitting 3 folds for each of 9 candidates, totalling 27 fits
0.5984879606490934
{'n_estimators': 100, 'max_depth': 4}


[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:    1.0s finished


**Пример использования pickle для сохранения обученных моделей.**

In [21]:
import pickle
# must open in binary format to pickle
pickle.dump(xgb_grid, open("best_boston.pkl", "wb"))
clf = pickle.load(open("best_boston.pkl", "rb"))
print(np.allclose(xgb_grid.predict(X), clf.predict(X)))

True


**Ранняя остановка используется для того, чтобы прекратить обучение модели (градиентный спуск), если ошибка за несколько итераций не уменьшилась.**

In [24]:
from sklearn.datasets import load_digits
from  sklearn.cross_validation import train_test_split

digits = load_digits()

X = digits['data']
y = digits['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
clf = xgb.XGBClassifier()
clf.fit(X_train, y_train, early_stopping_rounds=10, eval_metric="merror",
        eval_set=[(X_test, y_test)])

[0]	validation_0-merror:0.168889
Will train until validation_0-merror hasn't improved in 10 rounds.
[1]	validation_0-merror:0.162222
[2]	validation_0-merror:0.151111
[3]	validation_0-merror:0.142222
[4]	validation_0-merror:0.131111
[5]	validation_0-merror:0.128889
[6]	validation_0-merror:0.124444
[7]	validation_0-merror:0.111111
[8]	validation_0-merror:0.113333
[9]	validation_0-merror:0.111111
[10]	validation_0-merror:0.111111
[11]	validation_0-merror:0.111111
[12]	validation_0-merror:0.106667
[13]	validation_0-merror:0.113333
[14]	validation_0-merror:0.108889
[15]	validation_0-merror:0.104444
[16]	validation_0-merror:0.102222
[17]	validation_0-merror:0.102222
[18]	validation_0-merror:0.102222
[19]	validation_0-merror:0.104444
[20]	validation_0-merror:0.104444
[21]	validation_0-merror:0.1
[22]	validation_0-merror:0.093333
[23]	validation_0-merror:0.086667
[24]	validation_0-merror:0.086667
[25]	validation_0-merror:0.084444
[26]	validation_0-merror:0.088889
[27]	validation_0-merror:0.086

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

## Полезные ссылки
- <a href="https://en.wikipedia.org/wiki/Boosting_(machine_learning)">Boosting</a>
- [Gradient boosting](https://en.wikipedia.org/wiki/Gradient_boosting)
- [Лекция](http://www.machinelearning.ru/wiki/images/c/cd/Voron-ML-Compositions-slides.pdf) К.В. Воронцова по композиционным методам классификации
- <a href="https://github.com/dmlc/xgboost">Xgboost</a>
- <a href="https://github.com/ChenglongChen/Kaggle_CrowdFlower">Обзор</a> решения победителя соревнования Kaggle "CrowdFlower" по предсказанию релевантности выдачи поисковика товаров. Решение на основе Xgboost